First set up code to visualize a sound form

In [ ]:
%pylab inline
import IPython.display as ipd
import librosa
from librosa import load, display
import glob
# import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
from common import read_from_pickle

In [ ]:
# You should change these paths according to the path of the files on your system.
PATH_TO_TRAIN_LABELS = "data/train/train.csv"
PATH_TO_TEST_LABELS = "data/test/test.csv"
PATH_TO_TRAIN_AUDIO_FILES = "data/train/wav/"
PATH_TO_TEST_AUDIO_FILES = "data/test/wav/"
PATH_TO_SUBMISSION = "submission/"
PATH_TO_PICKLE = "pickles/"

In [115]:
chroma = read_from_pickle(PATH_TO_PICKLE + "nn chroma train.pickle")
contrast = read_from_pickle(PATH_TO_PICKLE + "nn contrast train.pickle")
mel =  read_from_pickle(PATH_TO_PICKLE + "nn mel train.pickle")
tonnetz =  read_from_pickle(PATH_TO_PICKLE + "nn tonnetz train.pickle")
mfccs =  read_from_pickle(PATH_TO_PICKLE + "nn mfccs train.pickle")

chroma_test = read_from_pickle(PATH_TO_PICKLE + "nn chroma test.pickle")
contrast_test = read_from_pickle(PATH_TO_PICKLE + "nn contrast test.pickle")
mel_test =  read_from_pickle(PATH_TO_PICKLE + "nn mel test.pickle")
tonnetz_test =  read_from_pickle(PATH_TO_PICKLE + "nn tonnetz test.pickle")
mfccs_test =  read_from_pickle(PATH_TO_PICKLE + "nn mfccs test.pickle")

FileNotFoundError: [Errno 2] No such file or directory: 'pickles/nn mfccs train.pickle'

In [ ]:
chroma = chroma[:2]
contrast = contrast[:2]
mel = mel[:2]
tonnetz = tonnetz[:2]
#mfccs = mfccs[:2]

chroma_test = chroma_test[:2]
contrast_test = contrast_test[:2]
mel_test = mel_test[:2]
tonnetz_test = tonnetz_test[:2]
# mfccs_test = mfccs_test[:2]

In [ ]:
train = pd.concat([chroma,mel,contrast,tonnetz], axis = 1)
test = pd.concat([chroma_test,mel_test,contrast_test,tonnetz_test], axis = 1)

In [114]:
train.columns = ['chroma', 'Class', 'mel', 'Class', 'contrast', 'Class', 'tonnetz', 'Class']
#train['feature'] = train['chroma'] + train['mel']

print(train['chroma'][0])
print(train['mel'][0])

[0.79807078 0.84270708 0.8235438  0.81167137 0.82789256 0.84835391
 0.86723692 0.82401034 0.80902169 0.77104424 0.76811742 0.79589275]
[5.87708541e+00 2.03388998e+01 5.20855222e+01 3.82289120e+01
 1.04353662e+01 9.48910564e+00 1.40696703e+01 1.35309566e+01
 9.39499475e+00 9.96546437e+00 5.37755803e+00 3.79177272e+00
 3.71443543e+00 2.82228024e+00 1.61042534e+00 1.56576814e+00
 9.82303401e-01 1.23218593e+00 1.76910114e+00 2.17238823e+00
 2.77292466e+00 3.52536819e+00 4.57555016e+00 5.85364322e+00
 5.62706726e+00 4.19326163e+00 4.34049825e+00 4.35999220e+00
 4.52208382e+00 4.63111460e+00 4.12360476e+00 4.10931668e+00
 4.67110659e+00 2.93461494e+00 2.37408304e+00 3.21437430e+00
 3.68181554e+00 2.84264293e+00 2.08246911e+00 3.19499744e+00
 2.68539929e+00 3.65215426e+00 3.59345507e+00 3.19026252e+00
 1.83367031e+00 1.51010177e+00 1.37544608e+00 1.64178969e+00
 1.90366473e+00 1.64655914e+00 1.60330134e+00 2.41138681e+00
 1.61818166e+00 1.30545541e+00 1.23210898e+00 1.32470569e+00
 1.10737082

In [104]:
len(A)

4

In [ ]:
X = np.concatenate((train['chroma'],train['mel'],train['contrast'],train['tonnetz']), axis = 0)

In [ ]:
print(train_features)

In [ ]:
# Renaming the columns to singnify what they mean helps with documentation,
# and also helps you keep track of them later on.
train_features.columns = ['feature','label']
# train_features.head()

In [ ]:
# this library helps us convert string labels into easy to handle encoded labels.
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

In [ ]:
X = np.array(train_features.feature.tolist())
Y = np.array(train_features.label.tolist())
lb = LabelEncoder()
# Since labels are categories they dont inherently have an order amongst themselves.
# For example, Apples > oranges does not make any sense. So to madel such categorical 
# variables, we can convert them to one hot vectors.
Y = to_categorical(lb.fit_transform(Y))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [ ]:
number_of_labels = Y.shape[1]
filter_size = 2

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [ ]:
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [ ]:
model.add(Dense(number_of_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer='adam')


In [ ]:
model.fit(X,Y, batch_size=32, epochs=50)

In [ ]:
def test_parser(row):
    global test_error_count
    global test_error_labels
    path_to_wav_files = PATH_TO_TEST_AUDIO_FILES
    file_path = path_to_wav_files + str(row.ID) + ".wav"
    try:
        data, sampling_rate = librosa.load(file_path, res_type='kaiser_fast')
        mfccs = np.mean( librosa.feature.mfcc(y = data, sr = sampling_rate, n_mfcc= 40).T, axis=0)
    except Exception as ex:
        test_error_count += 1
        test_error_labels.append(row.ID)
        return pd.Series([0]*40)
    features = mfccs
    return pd.Series(features)

In [ ]:
#try:
test_features = test.apply(test_parser,axis=1, reduce = True)
print("%d samples had errors while parsing" % test_error_count)
print("Errorneous samples", test_error_labels)
#except Exception as ex:
 #   print(ex)

In [ ]:
X_test = test_features
# X_test

In [ ]:
test_labels = model.predict(X_test, batch_size=32)

In [ ]:
test_labels_strings = lb.inverse_transform(test_labels.argmax(axis=1))
# test_labels_strings

In [ ]:
test['Class'] = test_labels_strings

In [ ]:
test.to_csv(PATH_TO_SUBMISSION + "sub_nn_56.csv",index=None)

This approach gives 56% accuracy with the above setup.